VGG 1

In [1]:
# Loading the dataset
import shutil
shutil.unpack_archive('/content/Dataset.zip', 'FinalDS')

In [2]:
# Importing Necessary Libraries

import matplotlib.pyplot as plt
import numpy as np
import os
import time
import PIL


In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Define input shape
input_shape = (640, 640, 3)
target_shape = (224, 224, 3)

# Define custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.data = ImageFolder(root_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]
        return img, label

# Data transformations
data_transform = transforms.Compose([
    transforms.Resize(target_shape[:2]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load training and testing data
train_dataset = CustomDataset('/content/FinalDS/Dataset/Train', transform=data_transform)
test_dataset = CustomDataset('/content/FinalDS/Dataset/Test', transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define VGG model with one block
class VGG1Block(nn.Module):
    def __init__(self):
        super(VGG1Block, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * (target_shape[0] // 2) * (target_shape[1] // 2), 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Create the model
vgg_model_1_block = VGG1Block()

# Set up TensorBoard
log_dir = "logsVGG1/"
writer = SummaryWriter(log_dir=log_dir)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(vgg_model_1_block.parameters(), lr=0.001)

# Training loop
for epoch in range(3):
    vgg_model_1_block.train()
    print("This is Epoch Number", epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        total_correct = 0
        total_samples = 0
        optimizer.zero_grad()
        output = vgg_model_1_block(data)
        loss = criterion(output, target.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        predicted = (output > 0.5).float()  # Convert probabilities to binary predictions
        total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
        total_samples += target.size(0)
        train_accuracy = total_correct / total_samples
        print("The Train Accuracy in epoch", epoch, "is", train_accuracy)

        # Log loss to TensorBoard
        writer.add_scalar("TrainLoss/Iter", loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar("TrainAccuracy/Iter", train_accuracy, epoch * len(train_loader) + batch_idx)

    # Validation
    vgg_model_1_block.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data, target in test_loader:
            output = vgg_model_1_block(data)
            predicted = (output > 0.5).float()
            total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
            total_samples += target.size(0)

        accuracy = total_correct / total_samples
        writer.add_scalar("TestAccuracy/Iter", accuracy, epoch)

# Close the SummaryWriter
writer.close()

print("Training completed!")


This is Epoch Number 0
The Train Accuracy in epoch 0 is 0.75
The Train Accuracy in epoch 0 is 0.125
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.25
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.25
This is Epoch Number 1
The Train Accuracy in epoch 1 is 0.25
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0.5
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0

# VGG 3

In [4]:
class VGG3Block(nn.Module):
    def __init__(self):
        super(VGG3Block, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * (target_shape[0] // 8) * (target_shape[1] // 8), 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Create the model
vgg_model_3_block = VGG3Block()

# Set up TensorBoard
log_dir = "logsVGG3/"
writer = SummaryWriter(log_dir=log_dir)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(vgg_model_3_block.parameters(), lr=0.001)

# Training loop
for epoch in range(3):
    vgg_model_3_block.train()
    print("This is Epoch Number", epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        total_correct = 0
        total_samples = 0
        optimizer.zero_grad()
        output = vgg_model_3_block(data)
        loss = criterion(output, target.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        predicted = (output > 0.5).float()  # Convert probabilities to binary predictions
        total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
        total_samples += target.size(0)
        train_accuracy = total_correct / total_samples
        print("The Train Accuracy in epoch", epoch, "is", train_accuracy)

        # Log loss to TensorBoard
        writer.add_scalar("TrainLoss/Iter", loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar("TrainAccuracy/Iter", train_accuracy, epoch * len(train_loader) + batch_idx)

    # Validation
    vgg_model_3_block.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data, target in test_loader:
            output = vgg_model_3_block(data)
            predicted = (output > 0.5).float()
            total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
            total_samples += target.size(0)

        accuracy = total_correct / total_samples
        writer.add_scalar("TestAccuracy/Iter", accuracy, epoch)

# Close the SummaryWriter
writer.close()

print("Training completed!")


This is Epoch Number 0
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.75
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.125
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.875
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.875
The Train Accuracy in epoch 0 is 0.0
The Train Accuracy in epoch 0 is 0.5
This is Epoch Number 1
The Train Accuracy in epoch 1 is 0.125
The Train Accuracy in epoch 1 is 0.5
The Train Accuracy in epoch 1 is 0.375
The Train Accuracy in epoch 1 is 0.25
The Train Accuracy in epoch 1 is 0

# VGG 3 with Augmentation

In [5]:
data_transform = transforms.Compose([
    transforms.RandomCrop(target_shape[:2]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load training and testing data
train_dataset = CustomDataset('/content/FinalDS/Dataset/Train', transform=data_transform)
test_dataset = CustomDataset('/content/FinalDS/Dataset/Test', transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

class VGG3Block(nn.Module):
    def __init__(self):
        super(VGG3Block, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * (target_shape[0] // 8) * (target_shape[1] // 8), 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Create the model
vgg_model_3_block = VGG3Block()

# Set up TensorBoard
log_dir = "logsVGG3AUG/"
writer = SummaryWriter(log_dir=log_dir)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(vgg_model_3_block.parameters(), lr=0.001)

# Training loop
for epoch in range(3):
    vgg_model_3_block.train()
    print("This is Epoch Number", epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        total_correct = 0
        total_samples = 0
        optimizer.zero_grad()
        output = vgg_model_3_block(data)
        loss = criterion(output, target.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        predicted = (output > 0.5).float()  # Convert probabilities to binary predictions
        total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
        total_samples += target.size(0)
        train_accuracy = total_correct / total_samples
        print("The Train Accuracy in epoch", epoch, "is", train_accuracy)

        # Log loss to TensorBoard
        writer.add_scalar("TrainLoss/Iter", loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar("TrainAccuracy/Iter", train_accuracy, epoch * len(train_loader) + batch_idx)

    # Validation
    vgg_model_3_block.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data, target in test_loader:
            output = vgg_model_3_block(data)
            predicted = (output > 0.5).float()
            total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
            total_samples += target.size(0)

        accuracy = total_correct / total_samples
        writer.add_scalar("TestAccuracy/Iter", accuracy, epoch)

# Close the SummaryWriter
writer.close()

print("Training completed!")


This is Epoch Number 0
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.875
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.875
The Train Accuracy in epoch 0 is 0.75
The Train Accuracy in epoch 0 is 0.875
The Train Accuracy in epoch 0 is 0.75
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.75
The Train Accuracy in epoch 0 is 0.875
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.75
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 0.625
This is Epoch Number 1
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0.75


# VGG 16

In [6]:
data_transform = transforms.Compose([
    transforms.RandomCrop(target_shape[:2]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load training and testing data
train_dataset = CustomDataset('/content/FinalDS/Dataset/Train', transform=data_transform)
test_dataset = CustomDataset('/content/FinalDS/Dataset/Test', transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

class VGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# Create the model
vgg16_model = VGG16()

# Set up TensorBoard
log_dir = "logsVGG16/"
writer = SummaryWriter(log_dir=log_dir)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(vgg16_model.parameters(), lr=0.001)

# Training loop
for epoch in range(3):
    vgg16_model.train()
    print("This is Epoch Number", epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        total_correct = 0
        total_samples = 0
        optimizer.zero_grad()
        output = vgg16_model(data)
        predicted = torch.tensor(torch.argmax(output, axis = 1).reshape((8,1)).float(), requires_grad=True)
        loss = criterion(predicted, target.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
        total_samples += target.size(0)
        train_accuracy = total_correct / total_samples
        print("The Train Accuracy in epoch", epoch, "is", train_accuracy)

        # Log loss to TensorBoard
        writer.add_scalar("TrainLoss/Iter", loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar("TrainAccuracy/Iter", train_accuracy, epoch * len(train_loader) + batch_idx)

    # Validation
    vgg16_model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data, target in test_loader:
            output = vgg16_model(data)
            predicted = (output > 0.5).float()
            total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
            total_samples += target.size(0)

        accuracy = total_correct / total_samples
        writer.add_scalar("TestAccuracy/Iter", accuracy, epoch)

# Close the SummaryWriter
writer.close()

print("Training completed!")


This is Epoch Number 0


<ipython-input-6-ffef49cc58a4>:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted = torch.tensor(torch.argmax(output, axis = 1).reshape((8,1)).float(), requires_grad=True)


The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.0
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.25
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 0.625
The Train Accuracy in epoch 0 is 1.0
The Train Accuracy in epoch 0 is 0.5
The Train Accuracy in epoch 0 is 0.25
The Train Accuracy in epoch 0 is 0.375
The Train Accuracy in epoch 0 is 0.625
This is Epoch Number 1
The Train Accuracy in epoch 1 is 0.5
The Train Accuracy in epoch 1 is 0.875
The Train Accuracy in epoch 1 is 0.625
The Train Accuracy in epoch 1 is 0.375
The Train Accuracy in epoch 1 is 0.875
The Train Accuracy in epoc

# VGG 16 with preprocessed layers

In [9]:
data_transform = transforms.Compose([
    transforms.RandomCrop(target_shape[:2]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load training and testing data
train_dataset = CustomDataset('/content/FinalDS/Dataset/Train', transform=data_transform)
test_dataset = CustomDataset('/content/FinalDS/Dataset/Test', transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


import torch
import torch.nn as nn
import torchvision.models as models

def create_transfer_model(input_shape, num_classes):
    # Load pre-trained VGG-16 model (excluding top layers)
    base_model = models.vgg16(pretrained=True)
    for param in base_model.parameters():
        param.requires_grad = False

    # Modify the classifier layers
    base_model.classifier = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
        nn.Sigmoid()
    )

    return base_model




# Example usage
input_shape = (3, 224, 224)  # Assuming RGB images of size 224x224
num_classes = 2  # Change this to the actual number of classes in your task
transfer_model = create_transfer_model(input_shape, num_classes)
print(transfer_model)



# Set up TensorBoard
log_dir = "logsVGG16PRE/"
writer = SummaryWriter(log_dir=log_dir)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(transfer_model.parameters(), lr=0.001)

# Training loop
for epoch in range(3):
    transfer_model.train()
    print("This is Epoch Number", epoch)
    for batch_idx, (data, target) in enumerate(train_loader):
        output = vgg16_model(data)
        predicted = torch.argmax(output, axis = 1).reshape((8,1)).float().clone().detach().requires_grad_(True)
        loss = criterion(predicted, target.float().unsqueeze(1))
        total_correct = 0
        total_samples = 0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_correct += (predicted == target.float().unsqueeze(1)).sum().item()
        total_samples += target.size(0)
        train_accuracy = total_correct / total_samples
        print("The Train Accuracy in epoch", epoch, "is", train_accuracy)
        # Log loss to TensorBoard
        writer.add_scalar("TrainLoss/Iter", loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar("TrainAccuracy/Iter", train_accuracy, epoch * len(train_loader) + batch_idx)

    # Validation
    transfer_model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data, target in test_loader:
            output = transfer_model(data)
            predicted_probs = torch.sigmoid(output)  # Apply sigmoid activation for binary classification
            predicted_labels = (predicted_probs > 0.5).float()
            total_correct += (predicted_labels == target.float().unsqueeze(1)).sum().item()
            total_samples += target.size(0)

        accuracy = total_correct / total_samples
        writer.add_scalar("TestAccuracy/Iter", accuracy, epoch)

# Close the SummaryWriter
writer.close()

print("Training completed!")


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1